<a href="https://colab.research.google.com/github/1900690/kyouyu/blob/main/yolov8.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">

  <a href="https://ultralytics.com/yolov8" target="_blank">
    <img width="1024", src="https://raw.githubusercontent.com/ultralytics/assets/main/yolov8/banner-yolov8.png"></a>


<br>
  <a href="https://console.paperspace.com/github/ultralytics/ultralytics"><img src="https://assets.paperspace.io/img/gradient-badge.svg" alt="Run on Gradient"/></a>
  <a href="https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  <a href="https://www.kaggle.com/ultralytics/yolov8"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>
<br>

Welcome to the Ultralytics YOLOv8 🚀 notebook! <a href="https://github.com/ultralytics/ultralytics">YOLOv8</a> is the latest version of the YOLO (You Only Look Once) object detection and image segmentation model developed by <a href="https://ultralytics.com">Ultralytics</a>. This notebook serves as the starting point for exploring the various resources available to help you get started with YOLOv8 and understand its features and capabilities.

The YOLOv8 models are designed to be fast, accurate, and easy to use, making them an excellent choice for a wide range of object detection and image segmentation tasks. They can be trained on large datasets and are capable of running on a variety of hardware platforms, from CPUs to GPUs.

Whether you are a seasoned machine learning practitioner or new to the field, we hope that the resources in this notebook will help you get the most out of YOLOv8. Please feel free to browse the <a href="https://docs.ultralytics.com/">YOLOv8 Docs</a> and reach out to us with any questions or feedback.

</div>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt) and check PyTorch and GPU.

In [2]:
# Pip install method (recommended)
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.6 🚀 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.0/78.2 GB disk)


In [ ]:
# Git clone method (for development)
!git clone https://github.com/ultralytics/ultralytics
%pip install -qe ultralytics

Cloning into 'ultralytics'...
remote: Enumerating objects: 3201, done.
remote: Counting objects: 100% (928/928), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 3201 (delta 780), reused 740 (delta 682), pack-reused 2273
Receiving objects: 100% (3201/3201), 2.41 MiB | 15.60 MiB/s, done.
Resolving deltas: 100% (2148/2148), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.7 MB/s eta 0:00:00


In [ ]:
import shutil
import os
from sklearn.model_selection import train_test_split

if not os.path.exists('/content/annotations'):
  shutil.unpack_archive('/content/drive/MyDrive/mite_annotations.zip', '/content/annotations')
if not os.path.exists('/content/originals'):
  shutil.unpack_archive('/content/drive/MyDrive/mite_original.zip', '/content/originals')
if not os.path.exists('/content/annotations_not_mite'):
  shutil.unpack_archive('/content/drive/MyDrive/not_mite_annotations.zip', '/content/annotations_not_mite')
if not os.path.exists('/content/originals_not_mite'):
  shutil.unpack_archive('/content/drive/MyDrive/not_mite_original.zip', '/content/originals_not_mite')

In [ ]:
#画像サイズを640にアップサンプリング（アノテーションはそのままでよい）
import cv2
import glob
import os

if not os.path.exists('/content/originals_upsamples/'):
  os.makedirs('/content/originals_upsamples/')
if not os.path.exists('/content/originals_upsamples_not_mite/'):
  os.makedirs('/content/originals_upsamples_not_mite/')

originals=('/content/originals/*')
read_files = glob.glob(originals)
originals_not_mite=('/content/originals_not_mite/*')
read_files_not_mite = glob.glob(originals_not_mite)

for imgpass in read_files:
  # 読み込む画像を選択
  img = cv2.imread(imgpass)
  # サイズ設定｜cv2では(幅、高さ）の順で数値を設定
  size = (640,640) 
  # 画像拡大・縮小 オプションで拡大計算式変更可能
  img_inter_area  = cv2.resize(img,size,interpolation = cv2.INTER_LINEAR) 
  #保存
  cv2.imwrite('/content/originals_upsamples/'+os.path.split(imgpass)[1], img_inter_area)
for imgpass in read_files_not_mite:
  # 読み込む画像を選択
  img = cv2.imread(imgpass)
  # サイズ設定｜cv2では(幅、高さ）の順で数値を設定
  size = (640,640) 
  # 画像拡大・縮小 オプションで拡大計算式変更可能
  img_inter_area  = cv2.resize(img,size,interpolation = cv2.INTER_LINEAR) 
  #保存
  cv2.imwrite('/content/originals_upsamples_not_mite/'+os.path.split(imgpass)[1], img_inter_area)

In [ ]:
import shutil
import os
from sklearn.model_selection import train_test_split

annotations=('/content/annotations')
originals=('/content/originals_upsamples')
annotations_not_mite=('/content/annotations_not_mite')
originals_not_mite=('/content/originals_upsamples_not_mite')

read_files_annotations = os.listdir(annotations)
read_files_annotations.sort()
read_files_originals= os.listdir(originals)
read_files_originals.sort()
read_files_annotations_not_mite = os.listdir(annotations_not_mite)
read_files_annotations_not_mite.sort()
read_files_originals_not_mite= os.listdir(originals_not_mite)
read_files_originals_not_mite.sort()

annotations_train, annotations_test, originals_train, originals_test = train_test_split(read_files_annotations,read_files_originals,test_size=0.2)
annotations_train_not_mite, annotations_test_not_mite, originals_train_not_mite, originals_test_not_mite = train_test_split(read_files_annotations_not_mite,read_files_originals_not_mite,test_size=0.00001)

In [ ]:
#contentバージョン
import os

if not os.path.exists('/content/ultralytics/dataset/mite/train/images'):
  os.makedirs('/content/ultralytics/dataset/mite/train/images')
  os.makedirs('/content/ultralytics/dataset/mite/train/labels')
  os.makedirs('/content/ultralytics/dataset/mite/valid/images')
  os.makedirs('/content/ultralytics/dataset/mite/valid/labels')

In [ ]:
#contentバージョン
import os
import math

for filename in annotations_train:
  shutil.copy( annotations+"/"+filename,'/content/ultralytics/dataset/mite/train/labels/')
for filename in annotations_test:
  shutil.copy( annotations+"/"+filename,'/content/ultralytics/dataset/mite/valid/labels')
for filename in originals_train:
  shutil.copy( originals+"/"+filename,'/content/ultralytics/dataset/mite/train/images')
for filename in originals_test:
  shutil.copy( originals+"/"+filename,'/content/ultralytics/dataset/mite/valid/images')
#ディレクトリのパスを指定
train_images = '/content/ultralytics/dataset/mite/train/images'
train_labels = '/content/ultralytics/dataset/mite/train/labels'
valid_images = '/content/ultralytics/dataset/mite/valid/images'
valid_labels = '/content/ultralytics/dataset/mite/valid/labels'
#ファイル数を出力
print("train/imagesは",sum(os.path.isfile(os.path.join(train_images, name)) for name in os.listdir(train_images)))
print("train/labelsは",sum(os.path.isfile(os.path.join(train_labels, name)) for name in os.listdir(train_labels)))
print("valid/imagesは",sum(os.path.isfile(os.path.join(valid_images, name)) for name in os.listdir(valid_images)))
print("valid/labelsは",sum(os.path.isfile(os.path.join(valid_labels, name)) for name in os.listdir(valid_labels)))
print("最大公約数は",math.gcd(sum(os.path.isfile(os.path.join(train_images, name)) for name in os.listdir(train_images)),sum(os.path.isfile(os.path.join(valid_images, name)) for name in os.listdir(valid_images))))

train/imagesは 7400
train/labelsは 7400
valid/imagesは 1851
valid/labelsは 1851


In [ ]:
import os
#os.remove(valid_images+"/"+originals_test[0])
#os.remove(valid_labels+"/"+annotations_test[0])

In [ ]:
import math
#ディレクトリのパスを指定
train_images = '/content/ultralytics/dataset/mite/train/images'
train_labels = '/content/ultralytics/dataset/mite/train/labels'
valid_images = '/content/ultralytics/dataset/mite/valid/images'
valid_labels = '/content/ultralytics/dataset/mite/valid/labels'
#ファイル数を出力
print("train/imagesは",sum(os.path.isfile(os.path.join(train_images, name)) for name in os.listdir(train_images)))
print("train/labelsは",sum(os.path.isfile(os.path.join(train_labels, name)) for name in os.listdir(train_labels)))
print("valid/imagesは",sum(os.path.isfile(os.path.join(valid_images, name)) for name in os.listdir(valid_images)))
print("valid/labelsは",sum(os.path.isfile(os.path.join(valid_labels, name)) for name in os.listdir(valid_labels)))
print("最大公約数は",math.gcd(sum(os.path.isfile(os.path.join(train_images, name)) for name in os.listdir(train_images)),sum(os.path.isfile(os.path.join(valid_images, name)) for name in os.listdir(valid_images))))

train/imagesは 7400
train/labelsは 7400
valid/imagesは 1850
valid/labelsは 1850
最大公約数は 1850


#★

In [ ]:
%%writefile /content/ultralytics/data.yaml
# COCO 2017 dataset http://cocodataset.org

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ./mite/train
val: ./mite/valid

# number of classes
nc: 1

# class names
names: ['mite']

Overwriting /content/ultralytics/data.yaml


In [3]:
import shutil
import os
os.mkdir('/content/ultralytics/')
shutil.unpack_archive('/content/drive/MyDrive/ultralytics.zip', '/content/ultralytics/')

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

ERROR: file:///content/ultralytics/ultralytics does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
#初めから
%cd '/content/drive/MyDrive/ultralytics'
#!yolo task=detect mode=train model=yolov8n.pt data=data.yaml epochs=30 imgsz=640
!yolo task=detect mode=train model="/content/drive/MyDrive/ultralytics/runs/detect/train4/weights/best.pt" data=data.yaml epochs=40 imgsz=640 conf=0.7

/content/drive/MyDrive/ultralytics
Ultralytics YOLOv8.0.6 🚀 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)
yolo/engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/ultralytics/runs/detect/train4/weights/best.pt, data=data.yaml, epochs=40, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, save_json=False, save_hybrid=False, conf=0.7, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, retina_masks=False, format=torchscript, keras=False, optimize=False, int8=False, dyn

In [ ]:
#途中から
%cd '/content/drive/MyDrive/ultralytics'
!yolo task=detect mode=train resume=True model="/content/drive/MyDrive/ultralytics/runs/detect/train/weights/last.pt" data=data.yaml epochs=40 imgsz=640 conf=0.7

In [ ]:
import shutil
import os
#os.mkdir('/content/ultralytics/')
shutil.make_archive('/content/drive/MyDrive/ultralytics', format='zip', root_dir='/content/ultralytics/')

'/content/drive/MyDrive/ultralytics.zip'

In [ ]:
import glob
originals=('/content/ultralytics/datasets/mite/train/images/*')
read_files = glob.glob(originals)

In [ ]:
read_files[0]

'/content/ultralytics/datasets/mite/train/images/DSC_0284_180_1668_1764_c0180_x-048_y0053_o.jpg'

[detectのオプション](https://zenn.dev/opamp/articles/51ee26445a1732)

In [ ]:
!yolo task=detect mode=predict model="/content/ultralytics/runs/detect/train4/weights/best.pt" conf=0.25 save_txt=True save_conf=True\
    source='/content/ultralytics/datasets/mite/train/images/DSC_0284_180_1668_1764_c0180_x-048_y0053_o.jpg'

Ultralytics YOLOv8.0.6 🚀 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)
Fusing layers... 
Model summary: 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
image 1/1 /content/ultralytics/datasets/mite/train/images/DSC_0284_180_1668_1764_c0180_x-048_y0053_o.jpg: 640x640 1 mite, 8.3ms
Speed: 0.5ms pre-process, 8.3ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/ultralytics/runs/detect/predict4
1 labels saved to /content/ultralytics/runs/detect/predict4/labels


In [ ]:
shutil.make_archive('/content/ultralytics', format='zip', root_dir='/content/ultralytics')

'/content/ultralytics.zip'

# 1. Predict

YOLOv8 may be used directly in the Command Line Interface (CLI) with a `yolo` command for a variety of tasks and modes and accepts additional arguments, i.e. `imgsz=640`. See a full list of available `yolo` [arguments](https://docs.ultralytics.com/config/) in the YOLOv8 [Docs](https://docs.ultralytics.com).


In [ ]:
# Run inference on an image with YOLOv8n
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://ultralytics.com/images/zidane.jpg'

100% 165k/165k [00:00<00:00, 12.0MB/s]
Ultralytics YOLOv8.0.5 🚀 Python-3.8.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)
100% 6.24M/6.24M [00:00<00:00, 58.7MB/s]

Fusing layers... 
YOLOv8n summary: 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
image 1/1 /content/zidane.jpg: 384x640 2 persons, 1 tie, 13.6ms
Speed: 0.4ms pre-process, 13.6ms inference, 52.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="https://user-images.githubusercontent.com/26833433/127574988-6a558aa1-d268-44b9-bf6b-62d4c605cc72.jpg" width="600">

# 2. Val
Validate a model's accuracy on the [COCO](https://cocodataset.org/#home) dataset's `val` or `test` splits. The latest YOLOv8 [models](https://github.com/ultralytics/ultralytics#models) are downloaded automatically the first time they are used.

※このデータで性能テストしろって指定できるみたい

In [ ]:
# Download COCO val
import torch
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip  # unzip

In [ ]:
# Validate YOLOv8n on COCO128 val
!yolo task=detect mode=val model=yolov8n.pt data=coco128.yaml

Ultralytics YOLOv8.0.5 🚀 Python-3.8.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)
100% 6.24M/6.24M [00:01<00:00, 6.32MB/s]

Fusing layers... 
YOLOv8n summary: 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

Dataset not found ⚠️, missing paths ['/datasets/coco128/images/train2017']
100% 6.66M/6.66M [00:00<00:00, 71.9MB/s]
Dataset download success ✅ (0.8s), saved to /datasets
100% 755k/755k [00:00<00:00, 44.6MB/s]
val: Scanning /datasets/coco128/labels/train2017... 126 images, 2 backgrounds, 0 corrupt: 100% 128/128 [00:00<00:00, 1451.73it/s]
val: New cache created: /datasets/coco128/labels/train2017.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 8/8 [00:05<00:00,  1.53it/s]
                   all        128        929       0.64      0.537      0.605      0.446
                person        128        254      0.797      0.677      0.764      0.538
               bicycle        128          6      0.514      0.333      0.

# 3. Train

<p align=""><a href="https://roboflow.com/?ref=ultralytics"><img width="1000" src="https://github.com/ultralytics/assets/raw/main/yolov8/banner-integrations.png"/></a></p>

Train YOLOv8 on [Detection](https://docs.ultralytics.com/tasks/detection/), [Segmentation](https://docs.ultralytics.com/tasks/segmentation/) and [Classification](https://docs.ultralytics.com/tasks/classification/) datasets.

In [ ]:
# Train YOLOv8n on COCO128 for 3 epochs
!yolo task=detect mode=train model=yolov8n.pt data=coco128.yaml epochs=3 imgsz=640

Ultralytics YOLOv8.0.5 🚀 Python-3.8.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco128.yaml, epochs=3, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, retina_masks=False, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, workspace=4, nms=False, lr0=0.01, lrf=0.01, momentum=

# 4. Export

Export a YOLOv8 model to any supported format with the `format` argument, i.e. `format=onnx`.

- 💡 ProTip: Export to [ONNX](https://onnx.ai/) or [OpenVINO](https://docs.openvino.ai/latest/index.html) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://developer.nvidia.com/tensorrt) for up to 5x GPU speedup.


| Format                                                                     | `format=`          | Model                     |
|----------------------------------------------------------------------------|--------------------|---------------------------|
| [PyTorch](https://pytorch.org/)                                            | -                  | `yolov8n.pt`              |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)                    | `torchscript`      | `yolov8n.torchscript`     |
| [ONNX](https://onnx.ai/)                                                   | `onnx`             | `yolov8n.onnx`            |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)                     | `openvino`         | `yolov8n_openvino_model/` |
| [TensorRT](https://developer.nvidia.com/tensorrt)                          | `engine`           | `yolov8n.engine`          |
| [CoreML](https://github.com/apple/coremltools)                             | `coreml`           | `yolov8n.mlmodel`         |
| [TensorFlow SavedModel](https://www.tensorflow.org/guide/saved_model)      | `saved_model`      | `yolov8n_saved_model/`    |
| [TensorFlow GraphDef](https://www.tensorflow.org/api_docs/python/tf/Graph) | `pb`               | `yolov8n.pb`              |
| [TensorFlow Lite](https://www.tensorflow.org/lite)                         | `tflite`           | `yolov8n.tflite`          |
| [TensorFlow Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | `edgetpu`          | `yolov8n_edgetpu.tflite`  |
| [TensorFlow.js](https://www.tensorflow.org/js)                             | `tfjs`             | `yolov8n_web_model/`      |
| [PaddlePaddle](https://github.com/PaddlePaddle)                            | `paddle`           | `yolov8n_paddle_model/`   |



In [ ]:
!yolo mode=export model=yolov8n.pt format=saved_model

Ultralytics YOLOv8.0.6 🚀 Python-3.8.16 torch-1.13.0+cu116 CPU
Fusing layers... 
YOLOv8n summary: 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from yolov8n.pt with output shape (1, 84, 8400) (6.2 MB)
Error executing job with overrides: ['mode=export', 'model=yolov8n.pt', 'format=saved_model']
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/ultralytics/yolo/cli.py", line 56, in cli
    func(cfg)
  File "/usr/local/lib/python3.8/dist-packages/hydra/main.py", line 79, in decorated_main
    return task_function(cfg_passthrough)
  File "/usr/local/lib/python3.8/dist-packages/ultralytics/yolo/engine/exporter.py", line 821, in export
    model.export(**cfg)
  File "/usr/local/lib/python3.8/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/ultralytics/yolo/engine/model.py", line 167, in export
    exporter(model=self.model)
  File

# 5. Python Usage

YOLOv8 was reimagined using Python-first principles for the most seamless Python YOLO experience yet. YOLOv8 models can be loaded from a trained checkpoint or created from scratch. Then methods are used to train, val, predict, and export the model. See a detailed Python usage examples in the YOLOv8 [Docs](https://docs.ultralytics.com/python/).

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')  # build a new model from scratch
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data='coco128.yaml', epochs=3)  # train the model
results = model.val()  # evaluate model performance on the validation set
results = model('https://ultralytics.com/images/bus.jpg')  # predict on an image
success = model.export(format='onnx')  # export the model to ONNX format

# 6. Tasks

YOLOv8 can train, val, predict and export models for the 3 primary tasks in vision AI: detection, segmentation and classification.

<img width="1024" src="https://user-images.githubusercontent.com/26833433/212094133-6bb8c21c-3d47-41df-a512-81c5931054ae.png">


## 1. Detection

YOLOv8 _detection_ models have no suffix and are the default YOLOv8 models, i.e. `yolov8n.pt` and are pretrained on COCO. See [Detection Docs](https://docs.ultralytics.com/tasks/detection/) for full details.


In [ ]:
# Load YOLOv8n, train it on COCO128 for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # load a pretrained YOLOv8n detection model
model.train(data='coco128.yaml', epochs=3)  # train the model
model('https://ultralytics.com/images/bus.jpg')  # predict on an image

## 2. Segmentation

YOLOv8 _segmentation_ models use the `-seg` suffix, i.e. `yolov8n-seg.pt` and are pretrained on COCO. See [Segmentation Docs](https://docs.ultralytics.com/tasks/segmentation/) for full details.


In [ ]:
# Load YOLOv8n-seg, train it on COCO128-seg for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')  # load a pretrained YOLOv8n segmentation model
model.train(data='coco128-seg.yaml', epochs=3)  # train the model
model('https://ultralytics.com/images/bus.jpg')  # predict on an image

## 3. Classification

YOLOv8 _classification_ models use the `-cls` suffix, i.e. `yolov8n-cls.pt` and are pretrained on ImageNet. See [Classification Docs](https://docs.ultralytics.com/tasks/classification/) for full details.


In [ ]:
# Load YOLOv8n-cls, train it on imagenette160 for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolov8n-cls.pt')  # load a pretrained YOLOv8n classification model
model.train(data='imagenette160', epochs=3)  # train the model
model('https://ultralytics.com/images/bus.jpg')  # predict on an image

# Appendix

Additional content below.

In [ ]:
# Run YOLOv8 tests (git clone install only)
!pytest ultralytics/tests